<h1> Red wine data analysis </h1>

<h2>Business Understanding</h2>

Through the business, we have been asked to examine the following questions:

<ol>
<li>In hoeverre is de score van een Portugese Red te voorspellen op basis van de chemische kenmerken?</li>
<li>Zelf te bepalen: denk aan andere type wijnen of andere landen.</li>
<li>Zelf te bepalen: denk bijvoorbeeld aan het clusteren op basis van de chemische kenmerken waarmee het type druif of de regio bepaald kan worden. Of kun je logische clusters vinden van topwijnen, doordrinkwijnen en bocht?</li>
</ol>

<i>TODO(m-jeu): Either translate these questions to english, or change the language of the rest of the document to dutch. This also goes for the dutch list in 'Data Understanding'.</i>

Currently, the exact nature of 'the business' and their desires datascience-wise is unknown to us. This requires further investigation.

<h2> Data Understanding </h2>

The business tells us the most important variables in the dataset are:

<ol>
<li>Herkomst van de wijn en type druif.</li>
<li>Review van de wijn, inclusief naam van de sommelier en de score op een schaal van 1 tot 100.</li>
<li>De uitkomsten van chemische tests op 11 waarde (waaronder suikergehalte, pH, alcoholgehalte, et cetera).</li>



We import some libraries and the dataset to examine the data through code.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
dataset = pd.read_csv("redwine.csv", sep=";")

<h2>Data Preparation</h2>